In [19]:
import pandas as pd
# 读取肥胖数据
df_obesity = pd.read_csv('Obesity among adults by country/obesity-cleaned.csv')
df_obesity = df_obesity.drop(df_obesity.columns[0], axis=1)
df_obesity = df_obesity.drop(df_obesity.columns[3], axis=1)
df_obesity.head()

,Country,Year,Obesity (%)
0,Afghanistan,1975,0.5 [0.2-1.1]
1,Afghanistan,1975,0.2 [0.0-0.6]
2,Afghanistan,1975,0.8 [0.2-2.0]
3,Afghanistan,1976,0.5 [0.2-1.1]
4,Afghanistan,1976,0.2 [0.0-0.7]


In [20]:
# 读取国家代码表
df_CountryCode = pd.read_csv('Extra data needed/wikipedia-iso-country-codes.csv')
df_CountryCode = df_CountryCode.drop(['Alpha-2 code', 'Numeric code', 'ISO 3166-2'], axis=1)
df_CountryCode.head()

,English short name lower case,Alpha-3 code
0,Zimbabwe,ZWE
1,Zambia,ZMB
2,Yemen,YEM
3,Western Sahara,ESH
4,Wallis and Futuna,WLF


In [21]:
# 将肥胖数据中的国家改为国家代码
df_obesity_merged = pd.merge(df_obesity, df_CountryCode, left_on='Country', right_on='English short name lower case')
df_obesity_merged.drop(['Country', 'English short name lower case'], axis=1, inplace=True)
df_obesity_merged.rename(columns={'Alpha-3 code': 'Country_Code'}, inplace=True)
df_obesity = df_obesity_merged
df_obesity.head()

,Year,Obesity (%),Country_Code
0,1975,0.5 [0.2-1.1],AFG
1,1975,0.2 [0.0-0.6],AFG
2,1975,0.8 [0.2-2.0],AFG
3,1976,0.5 [0.2-1.1],AFG
4,1976,0.2 [0.0-0.7],AFG


In [25]:
#读取GDP数据
df_GDP = pd.read_csv('Country Economic Indicators/GDP.csv')
df_GDP = df_GDP[['LOCATION', 'TIME', 'Value']]
df_GDP = df_GDP.rename(columns={'LOCATION': 'Country_Code', 'TIME': 'Year','Value': 'GDP'})
df_GDP.head()

,Country_Code,Year,GDP
0,AUS,1960,25073.260
1,AUS,1961,25364.881
2,AUS,1962,27955.357
3,AUS,1963,30432.979
4,AUS,1964,32745.289


In [26]:
#读取排放数据
df_MtCO2 = pd.read_csv('Emissions by Country/GCB2022v27_MtCO2_flat.csv')
df_MtCO2 = df_MtCO2[['ISO 3166-1 alpha-3', 'Year', 'Total']]
df_MtCO2 = df_MtCO2.rename(columns={'ISO 3166-1 alpha-3': 'Country_Code','Total': 'MtCO2'})
df_MtCO2.head()

,Country_Code,Year,MtCO2
0,AFG,1750,0.0
1,AFG,1751,0.0
2,AFG,1752,0.0
3,AFG,1753,0.0
4,AFG,1754,0.0


In [27]:
# 清理数据：删除含有 0, None, 或空值的行
df_obesity = df_obesity[(df_obesity['Obesity (%)'] != 0) & (df_obesity['Obesity (%)'].notna())]
df_GDP = df_GDP[(df_GDP['GDP'] != 0) & (df_GDP['GDP'].notna())]
df_MtCO2 = df_MtCO2[(df_MtCO2['MtCO2'] != 0) & (df_MtCO2['MtCO2'].notna())]
# 合并数据框
df_merged = pd.merge(df_obesity, df_GDP, on=['Country_Code', 'Year'], how='inner')
df_merged = pd.merge(df_merged, df_MtCO2, on=['Country_Code', 'Year'], how='inner')
df_merged.head()

,Year,Obesity (%),Country_Code,GDP,MtCO2
0,1997,12.0 [8.9-15.6],ALB,8555.927000,1.543009
1,1997,12.0 [8.9-15.6],ALB,2726.202207,1.543009
2,1997,10.4 [6.2-15.4],ALB,8555.927000,1.543009
3,1997,10.4 [6.2-15.4],ALB,2726.202207,1.543009
4,1997,13.7 [9.2-18.8],ALB,8555.927000,1.543009
